<a href="https://colab.research.google.com/github/IvanDePivan/2AMM30-groep-2-component-1/blob/main/Flairmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining Group 2
Let's import everything here so we don't have to do this later

## Prerequisites

In [1]:
from google.colab import output
!pip uninstall -y torch
!pip install flair

import os
import random
import json
import re
import regex
import difflib
import srsly
import torch
from typing import List
from pathlib import Path

from flair.trainers import ModelTrainer
from flair.data import Corpus, Label, Relation, Sentence
from flair.datasets import ColumnCorpus, DataLoader
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TokenEmbeddings, TransformerWordEmbeddings, TransformerDocumentEmbeddings
from flair.models import SequenceTagger, RelationExtractor, RelationClassifier
from flair.models.relation_classifier_model import EncodedSentence

output.clear()

### Reformatting Label Studio output
We wrote this method for converting the output from Label Studio to the conll format, because the standard format had issues (e.g. No relation data was exported, tokenization had flaws, output used different delimiters)

In [ ]:
def write_conll(filename, text):
  output_file = Path(filename)
  output_file.parent.mkdir(exist_ok=True, parents=True)
  with open(output_file, "w") as file1:
    file1.write("\n\n".join(text))

In [ ]:
# Function for getting each token with their correct label and id (ids are used for relations)
def get_labels(file_item, key_field):
  rows = []
  position = 0
  next_label = 0
  # Sort the labels on start?
  labels = [item for item in file_item['annotations'][0]['result'] if item['type'] == 'labels']
  labels.sort(key=lambda x: x['value']['start'])
  item_count = 1
  b_count = 0
  ids = {} # count -> id
  tokens = [item for item in re.split(r'(\s+)|([\(,\.\)\\;:$*&"\-–])|(\')|(\'\w+)', file_item['data'][key_field]) if item]
  for i, item in enumerate(tokens):
    if re.search(r'\s+', item):
      position += len(item)
      continue
    start = position
    end = position + len(item)
    row = f'{item_count}\t{item}'
    label = '\tO'

    if len(labels) > next_label:
      if start >= labels[next_label]['value']['start']:
        # if end <= labels[next_label]['value']['end']:
          # the current item has a label, and might have a relation
          id = labels[next_label]["id"]
          ids[item_count] = id
          if end > labels[next_label]['value']['end']:
            difference = end - labels[next_label]['value']['end']
            end -= difference
            tokens.insert(i+1, item[-difference:])
            item = item[:-difference]
          if start == labels[next_label]['value']['start']:
            # Current Item is start of a label
            b_count += 1
            label = '\tB-' + labels[next_label]["value"]["labels"][0]

          else:
            # Current item is inside a label
            if '\tB-' + labels[next_label]["value"]["labels"][0] in rows[-1] or '\tI-' + labels[next_label]["value"]["labels"][0] in rows[-1]:
              label = '\tI-' + labels[next_label]["value"]["labels"][0]
            else:
              b_count += 1
              label = '\tB-' + labels[next_label]["value"]["labels"][0]

      if end >= labels[next_label]['value']['end']:
        next_label += 1


    row = row + label
    position += len(item)
    item_count += 1
    rows.append(row)
  # Sanity check on the labels, gives a warning if labels are disappearing.
  if len(labels) != b_count:
    print(f'labels: {len(labels)}, B-tags: {b_count}')
    print(labels)
    print(ids)
    print(f'missing label: {[item for item in labels if item["id"] not in ids.values()]}')
    print(rows)
  return rows, ids


In [ ]:
# This creates the relation format flair expects for each document
def get_relations(file_item, ids):
  keys = ids.keys()
  relations = [item for item in file_item['annotations'][0]['result'] if item['type'] == 'relation']
  bio_relations = []
  found_relations = 0
  for relation in relations:
    from_ids = []
    to_ids = []
    rel_type = relation['labels'][0]
    for key in keys:
      if ids[key] == relation['from_id']:
        from_ids.append(key)
      if ids[key] == relation['to_id']:
        to_ids.append(key)
    from_ids = [int(i) for i in from_ids]
    from_ids.sort()
    to_ids = [int(i) for i in to_ids]
    to_ids.sort()
    if len(from_ids) > 0 and len(to_ids) > 0:
      found_relations += 1
      bio_relations.append(f'{from_ids[0]};{from_ids[-1]};{to_ids[0]};{to_ids[-1]};{rel_type}')
  if found_relations != len(relations):
    print(f'found relations: {found_relations}, actual relations: {len(relations)}')
    print(relations)
    print(bio_relations)
    print(keys)
    print(ids)
  return bio_relations

In [ ]:
# creates the connl file using the expanded json file as input. This is reused for component 2
def make_conll(file, key_field='text', file_name='test_set'):
  conllu_file = []
  for item in file:
    doc = ''
    rows, ids = get_labels(item, key_field)
    relations = get_relations(item, ids)
    if relations and len(relations) > 0:
      doc += '# relations = ' + ('|'.join(relations))
    doc += '\n' + ('\n'.join(rows))
    conllu_file.append(doc)

  print(len(conllu_file))
  write_conll(f'{file_name}.conll', conllu_file)

In [ ]:
with open('test_set.json') as f:
  file = json.load(f)
  make_conll(file)

In [ ]:
# The transformed file is split up for train and dev data
# Test data is gathered from a separate json export from label studio, but it uses the same methods
# The split files are stored for convenience
with open('train_set.conll') as f:
  data = f.read()
  splits = data.split(2*os.linesep)
  print(len(splits))
  splits = sorted(splits, key=lambda x: random.random())

  train = splits[:-25]
  dev = splits[-25:]

  write_conll('train.conll', train)
  # write_conll('test.conll', test)
  write_conll('dev.conll', dev)

95


In [ ]:
# Sanity check on the gpu; some issues with flair not recognizing one of the GPUs from Colab
torch.cuda.is_available()

True

## Component 1

In [ ]:
# The corpus converts the BIO formatted tokens into a format Flair uses for training
# Here we can simply specify the folder where the files are and which columns they have
# Only supports
columns = {1: 'text', 2: 'ner'}
data_folder = ''

corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll',
                              dev_file='dev.conll')
print(f'Corpus size: {len(corpus.train)}')

In [ ]:
# tag to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)


In [ ]:

# embedding_types : List[TokenEmbeddings] = [
#         WordEmbeddings('glove'),
        ## other embeddings
        # ]
# embeddings : StackedEmbeddings = StackedEmbeddings(
#                                 embeddings=embedding_types)

embeddings = TransformerWordEmbeddings(
    model='distilbert-base-cased',
    layers="-1",
    subtoken_pooling='first',
    fine_tune=True,
    use_context=True,
    respect_document_boundaries=False,

)

tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,
        embeddings=embeddings,
        tag_dictionary=tag_dictionary,
        tag_type=tag_type,
        use_crf=False,
        use_rnn=False,
        reproject_embeddings=False,
        tag_format='BIO'
    )

In [ ]:
trainer : ModelTrainer = ModelTrainer(tagger, corpus)

trainer.fine_tune(
        'resources/taggers/ner',
        learning_rate=5e-05,
        mini_batch_size=16,
        max_epochs=100,
        embeddings_storage_mode='gpu', # cpu, gpu
        weight_decay=0.0,
    )


2023-10-23 09:38:11,444 ----------------------------------------------------------------------------------------------------
2023-10-23 09:38:11,454 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(28997, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_fea

100%|██████████| 2/2 [00:03<00:00,  1.51s/it]

2023-10-23 09:38:21,411 Evaluating as a multi-label problem: False
2023-10-23 09:38:21,478 DEV : loss 2.3361923694610596 - f1-score (micro avg)  0.0182
2023-10-23 09:38:21,495 ----------------------------------------------------------------------------------------------------


2023-10-23 09:38:21,990 epoch 2 - iter 1/5 - loss 2.40558857 - time (sec): 0.49 - samples/sec: 9416.41 - lr: 0.000006
2023-10-23 09:38:22,490 epoch 2 - iter 2/5 - loss 2.38843878 - time (sec): 0.99 - samples/sec: 9150.46 - lr: 0.000007
2023-10-23 09:38:22,963 epoch 2 - iter 3/5 - loss 2.37580171 - time (sec): 1.47 - samples/sec: 8857.16 - lr: 0.000008
2023-10-23 09:38:23,484 epoch 2 - iter 4/5 - loss 2.33474647 - time (sec): 1.99 - samples/sec: 8931.96 - lr: 0.000008
2023-10-23 09:38:23,789 epoch 2 - iter 5/5 - loss 2.30010249 - time (sec): 2.29 - samples/sec: 8751.81 - lr: 0.000009
2023-10-23 09:38:23,803 ----------------------------------------------------------------------------------------------------
2023-10-23 09:38:23,812 EPOCH 2 done: loss 2.3001 - lr 0.000009


100%|██████████| 2/2 [00:03<00:00,  1.69s/it]

...

2023-10-23 09:46:56,549 Evaluating as a multi-label problem: False
2023-10-23 09:46:56,573 DEV : loss 0.3232954740524292 - f1-score (micro avg)  0.8582


2023-10-23 09:46:57,328 ----------------------------------------------------------------------------------------------------
2023-10-23 09:46:57,334 Testing using last state of model ...


100%|██████████| 3/3 [00:02<00:00,  1.42it/s]

2023-10-23 09:46:59,469 Evaluating as a multi-label problem: False
2023-10-23 09:46:59,507 0.7849	0.8502	0.8162	0.6895
2023-10-23 09:46:59,511 
Results:
- F-score (micro) 0.8162
- F-score (macro) 0.7237
- Accuracy 0.6895

By class:
              precision    recall  f1-score   support

      Winner     0.7056    0.7710    0.7368       345
        Date     0.9728    0.9921    0.9823       252
 Nationality     0.7979    0.8750    0.8347       176
   Prizetype     0.7600    0.9268    0.8352        41
      Reason     0.2059    0.2593    0.2295        27

   micro avg     0.7849    0.8502    0.8162       841
   macro avg     0.6884    0.7648    0.7237       841
weighted avg     0.7916    0.8502    0.8194       841

2023-10-23 09:46:59,513 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8162100456621005,
 'dev_score_history': [0.018193094893529046,
  0.013029315960912051,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.3155216284987277,
  0.5126135216952573,
  0.5595959595959595,
  0.7158081705150978,
  0.7333333333333333,
  0.7896200185356812,
  0.7723502304147465,
  0.7992988606485539,
  0.7899159663865546,
  0.8388349514563107,
  0.828649138712602,
  0.8249763481551562,
  0.8181818181818183,
  0.8255915863277826,
  0.8299817184643511,
  0.8238482384823849,
  0.8469945355191256,
  0.850510677808728,
  0.8234265734265735,
  0.8407871198568873,
  0.8345323741007195,
  0.8430493273542601,
  0.855822550831793,
  0.853185595567867,
  0.8403967538322814,
  0.8539741219963033,
  0.8346595932802829,
  0.8191681735985534,
  0.831858407079646,
  0.860377358490566,
  0.8499550763701708,
  0.8504923903312445,
  0.8493919550982225,
  0.8566340160284951,
  0.8459383753501402,
  0.8398190045248868,
  0.850909090909091,
  0.8624192059095107,
  0.8476702508960574,
  0

In [ ]:
# load the trained model
model = SequenceTagger.load("resources/taggers/ner/final-model.pt")

# create example sentence
sentence = Sentence('Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer. ')

# predict the tags
model.predict(sentence)
print(sentence.to_tagged_string())

2023-10-18 10:08:04,163 SequenceTagger predicts: Dictionary with 11 tags: O, B-Winner, I-Winner, B-Date, I-Date, B-Nationality, I-Nationality, B-Prizetype, I-Prizetype, B-Reason, I-Reason
Sentence[79]: "Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer." → ["Barack Hussein Obama II"/Winner, "August 4, 1961"/Date, "American"/Nationality, "the United States"/Nationality, "he"/Winner, "Obama"/Winner, "U.S."/Nationality, "Illinois"/Nationality]


In [ ]:


relation_label_dict = corpus.make_label_dictionary(label_type="relation")
results_base_path = 'resources/relations/'
relation_model: RelationExtractor = RelationExtractor(
        embeddings=embeddings,
        label_dictionary=relation_label_dict,
        label_type="relation",
        entity_label_type="ner",
    )

trainer: ModelTrainer = ModelTrainer(relation_model, corpus)

trainer.train(
        results_base_path,
        learning_rate=5e-05,
        mini_batch_size=2,
        max_epochs=100,
        shuffle=True,
    )

2023-10-25 07:01:29,504 Computing label dictionary. Progress:


70it [00:00, 9517.06it/s]

2023-10-25 07:01:29,521 Dictionary created for label 'relation' with 7 values: has_won (seen 114 times), received_nobelprize_in (seen 111 times), received_nobelprize_for (seen 78 times), has_nationality (seen 70 times), born_on (seen 57 times), died_on (seen 29 times)


NameError: ignored

In [ ]:


relation_label_dict = corpus.make_label_dictionary("relation")
results_base_path = 'resources/relations/'

for i in range(100):
  print(relation_label_dict[0])


2023-10-25 07:18:43,542 Computing label dictionary. Progress:


70it [00:00, 5157.96it/s]

2023-10-25 07:18:43,580 Dictionary created for label 'relation' with 7 values: has_won (seen 114 times), received_nobelprize_in (seen 111 times), received_nobelprize_for (seen 78 times), has_nationality (seen 70 times), born_on (seen 57 times), died_on (seen 29 times)


TypeError: ignored

In [ ]:
relation_label_dict = corpus.make_label_dictionary(label_type="relation")
relation_dictionary = corpus.make_label_dictionary("relation")
doc_embeddings = TransformerDocumentEmbeddings(model="distilbert-base-uncased", layers="-1", fine_tune=True)
relation_model: RelationClassifier = RelationClassifier(
    embeddings =doc_embeddings,
    label_dictionary = relation_dictionary,
    label_type="relation",
    entity_label_types="ner",
    entity_pair_labels={  # Define valid entity pair combinations, used as relation candidates
        ("Winner", "Nationality"),
        ("Winner", "Date"),
        ("Winner", "Prizetype"),
        ("Winner", "Reason"),

    },
    zero_tag_value='',
    allow_unk_tag=False,
)

2023-10-23 10:03:20,155 Computing label dictionary. Progress:


70it [00:00, 7128.32it/s]

2023-10-23 10:03:20,172 Dictionary created for label 'relation' with 7 values: has_won (seen 114 times), received_nobelprize_in (seen 111 times), received_nobelprize_for (seen 78 times), has_nationality (seen 70 times), born_on (seen 57 times), died_on (seen 29 times)
2023-10-23 10:03:20,175 Computing label dictionary. Progress:



70it [00:00, 16680.94it/s]

2023-10-23 10:03:20,187 Dictionary created for label 'relation' with 7 values: has_won (seen 114 times), received_nobelprize_in (seen 111 times), received_nobelprize_for (seen 78 times), has_nationality (seen 70 times), born_on (seen 57 times), died_on (seen 29 times)


In [ ]:
trainer: ModelTrainer = ModelTrainer(model=relation_model, corpus=relation_model.transform_corpus(corpus))


In [ ]:
trainer.fine_tune(
    'resources/relations/',
    max_epochs=20,
    learning_rate=4e-5,
    mini_batch_size=8,
    main_evaluation_metric=("macro avg", "f1-score"),
    shuffle=True,
    embeddings_storage_mode='gpu',
)

2023-10-23 10:16:13,234 ----------------------------------------------------------------------------------------------------
2023-10-23 10:16:13,240 Model: "RelationClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768,

100%|██████████| 373/373 [00:26<00:00, 13.86it/s]

2023-10-23 10:18:49,424 Evaluating as a multi-label problem: False
2023-10-23 10:18:49,454 DEV : loss 0.32806316018104553 - f1-score (macro avg)  0.1391


2023-10-23 10:18:49,881 ----------------------------------------------------------------------------------------------------
2023-10-23 10:19:05,500 epoch 2 - iter 104/1041 - loss 0.28892615 - time (sec): 15.62 - samples/sec: 53.27 - lr: 0.000022
2023-10-23 10:19:21,385 epoch 2 - iter 208/1041 - loss 0.24636098 - time (sec): 31.50 - samples/sec: 52.82 - lr: 0.000024
2023-10-23 10:19:37,209 epoch 2 - iter 312/1041 - loss 0.23749319 - time (sec): 47.33 - samples/sec: 52.74 - lr: 0.000026
2023-10-23 10:19:52,924 epoch 2 - iter 416/1041 - loss 0.22281695 - time (sec): 63.04 - samples/sec: 52.79 - lr: 0.000028
2023-10-23 10:20:08,706 epoch 2 - iter 520/1041 - loss 0.20695863 - time (sec): 78.82 - samples/sec: 52.78 - lr: 0.000030
2023-10-23 10:20:24,498 epoch 2 - iter 624/1041 - loss 0.20421484 - time (sec): 94.62 - samples/sec: 52.76 - lr: 0.000032
2023-10-23 10:20:40,314 epoch 2 - iter 728/1041 - loss 0.19837392 - time (sec): 110.43 - samples/sec: 52.74 - lr: 0.000034
2023-10-23 10:20:56,

100%|██████████| 373/373 [00:26<00:00, 14.21it/s]

...

2023-10-23 11:17:09,217 Evaluating as a multi-label problem: False
2023-10-23 11:17:09,248 DEV : loss 0.23136982321739197 - f1-score (macro avg)  0.827
2023-10-23 11:17:10,274 ----------------------------------------------------------------------------------------------------
2023-10-23 11:17:10,276 Testing using last state of model ...


100%|██████████| 621/621 [00:45<00:00, 13.50it/s]

2023-10-23 11:17:56,324 Evaluating as a multi-label problem: False


2023-10-23 11:17:56,367 0.9698	0.9698	0.9698	0.9698
2023-10-23 11:17:56,368 
Results:
- F-score (micro) 0.9698
- F-score (macro) 0.76
- Accuracy 0.9698

By class:
                         precision    recall  f1-score   support

                            0.9891    0.9795    0.9843      4722
                has_won     0.4948    0.8571    0.6275        56
 received_nobelprize_in     0.7800    0.6842    0.7290        57
        has_nationality     0.7234    0.8293    0.7727        41
received_nobelprize_for     0.9000    0.6279    0.7397        43
                born_on     0.7105    0.8710    0.7826        31
                died_on     0.5200    1.0000    0.6842        13

               accuracy                         0.9698      4963
              macro avg     0.7311    0.8356    0.7600      4963
           weighted avg     0.9752    0.9698    0.9714      4963

2023-10-23 11:17:56,370 -----------------------------------------------------------------------------------------------

{'test_score': 0.759992490515059,
 'dev_score_history': [0.1390958035253337,
  0.6160874458762523,
  0.707193080150119,
  0.7862231454261847,
  0.8736109075274436,
  0.8431359510490344,
  0.8030713406771318,
  0.8519476743620954,
  0.86641966964504,
  0.8121858848860871,
  0.7866049532201755,
  0.7885083381019988,
  0.8473148562439665,
  0.7738092576039011,
  0.7912102509766569,
  0.827115409731006,
  0.8334859654930178,
  0.8238420679965188,
  0.8255898902125047,
  0.8269938965332065],
 'train_loss_history': [0.46602304046316584,
  0.18917429960275162,
  0.11208232270924051,
  0.08409133076309062,
  0.06015949468311575,
  0.04139665268956039,
  0.042408625662549736,
  0.02972297203058366,
  0.028851280507331743,
  0.015085002562761459,
  0.01795085951991243,
  0.012832369261971335,
  0.008864204655039811,
  0.00667855525070142,
  0.00915724442094882,
  0.006436865837237233,
  0.003423224463244922,
  0.0026045638967363022,
  0.0021427500815946144,
  0.0014627958245493652],
 'dev_loss_h

In [ ]:
loaded_ner = SequenceTagger.load("/content/drive/MyDrive/enwiki20230820/component1/ner/final-model.pt")
loaded_re: RelationClassifier = RelationClassifier.load('/content/drive/MyDrive/enwiki20230820/component1/relations/final-model.pt')

2023-10-30 12:42:28,880 SequenceTagger predicts: Dictionary with 11 tags: O, B-Winner, I-Winner, B-Date, I-Date, B-Nationality, I-Nationality, B-Prizetype, I-Prizetype, B-Reason, I-Reason


In [ ]:
sentence = Sentence('Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer. ')
loaded_ner.predict(sentence)
loaded_re.predict(sentence)


In [ ]:

# print(sentence)
# for item in sentence.get_labels():
  # label is item.value
  # if type(item.data_point) == Relation:
    # item:Relation
    # print(item.data_point.start_position, item.data_point.end_position, item.data_point.tag)
# for entity in sentence.get_labels('ner'):
#   e = entity
#   # print(e.to_dict(), e.shortstring, e.data_point.start_position, e.data_point.end_position)
#   print(entity.data_point.to_dict())
#   # print(entity.score, entity.value, entity)
# for entity in sentence.get_labels('relation'):
#   e = entity
#   # print(e.to_dict(), e.shortstring, e.data_point.start_position, e.data_point.end_position)
#   print(entity.data_point.to_dict())
#   # print(entity.score, entity.value, entity)

# srsly.write_json('sample.json', sentence.to_dict())

test_set = srsly.read_json('/content/drive/MyDrive/enwiki20230820/component1/val_set.json')
all_relations = {}
document_dicts = []
for document in test_set:
  sentence = Sentence(document["data"]["text"])
  loaded_ner.predict(sentence)
  loaded_re.predict(sentence)
  sent_dict = sentence.to_dict()
  # relation["labels"][0]["confidence"]
  all_relations[document["data"]["title"]] = [(relation["from_text"], relation["to_text"], relation["labels"][0]["value"]) for relation in sent_dict["relations"] if len(relation["labels"][0]["value"]) > 0]
  document_dicts.append(sentence.to_dict())


srsly.write_json('relations_test_set.json', all_relations)
srsly.write_json('test_document_dicts.json', document_dicts)


# Component 2
For component 2 the same model is used. This is finetuned to the new data.

In [ ]:
train_json = srsly.read_json('/content/drive/MyDrive/enwiki20230820/component2/train_data.json')
make_conll(train_json, key_field='Abstract_new', file_name='component2/train_set')

test_json = srsly.read_json('/content/drive/MyDrive/enwiki20230820/component2/test_data.json')
make_conll(test_json, key_field='Abstract_new', file_name='component2/test_set')

# Split the train file into train/dev split
with open('component2/train_set.conll') as f:
  data = f.read()
  splits = data.split(2*os.linesep)
  splits = sorted(splits, key=lambda x: random.random())

  train = splits[:90]
  dev = splits[90:]

  write_conll('component2/train.conll', train)
  write_conll('component2/dev.conll', dev)

with open('component2/test_set.conll') as f:
  data = f.read()
  splits = data.split(2*os.linesep)
  splits = sorted(splits, key=lambda x: random.random())

  write_conll('component2/test.conll', splits)


102
40


In [ ]:
columns = {1: 'text', 2: 'ner'}
data_folder = ''

corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='component2/train.conll',
                              test_file='component2/test.conll',
                              dev_file='component2/dev.conll')
print(f'Corpus size: {len(corpus.train)}')

2023-10-30 16:25:19,526 Reading data from .
2023-10-30 16:25:19,528 Train: component2/train.conll
2023-10-30 16:25:19,530 Dev: component2/dev.conll
2023-10-30 16:25:19,531 Test: component2/test.conll
Corpus size: 90


In [ ]:
# tag to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

embeddings = TransformerWordEmbeddings(
    model='distilbert-base-cased',
    layers="-1",
    subtoken_pooling='mean',
    fine_tune=True,
    use_context=True,
    respect_document_boundaries=False,
)

tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,
        embeddings=embeddings,
        tag_dictionary=tag_dictionary,
        tag_type=tag_type,
        use_crf=False,
        use_rnn=False,
        reproject_embeddings=False,
        tag_format='BIO'
    )

2023-10-30 16:25:31,040 Computing label dictionary. Progress:


0it [00:00, ?it/s]
90it [00:00, 4474.24it/s]

2023-10-30 16:25:31,073 Dictionary created for label 'ner' with 4 values: Component (seen 1138 times), Person (seen 360 times), Time (seen 119 times), Date (seen 119 times)


2023-10-30 16:25:32,508 SequenceTagger predicts: Dictionary with 9 tags: O, B-Component, I-Component, B-Person, I-Person, B-Time, I-Time, B-Date, I-Date


In [ ]:
trainer : ModelTrainer = ModelTrainer(tagger, corpus)

trainer.fine_tune(
        'component2/taggers/ner',
        learning_rate=5e-05,
        mini_batch_size=16,
        max_epochs=100,
        embeddings_storage_mode='gpu', # cpu, gpu
        weight_decay=0.0,
    )

2023-10-30 16:25:37,993 ----------------------------------------------------------------------------------------------------
2023-10-30 16:25:37,999 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(28997, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_fea

100%|██████████| 1/1 [00:00<00:00,  1.16it/s]

2023-10-30 16:25:56,993 DEV : loss 1.9528019428253174 - f1-score (micro avg)  0.0139
2023-10-30 16:25:56,999 ----------------------------------------------------------------------------------------------------


2023-10-30 16:25:59,998 epoch 2 - iter 1/6 - loss 2.08249749 - time (sec): 3.00 - samples/sec: 1389.74 - lr: 0.000005 - momentum: 0.000000
2023-10-30 16:26:02,597 epoch 2 - iter 2/6 - loss 2.06216805 - time (sec): 5.60 - samples/sec: 1402.78 - lr: 0.000005 - momentum: 0.000000
2023-10-30 16:26:06,299 epoch 2 - iter 3/6 - loss 2.02950072 - time (sec): 9.30 - samples/sec: 1299.45 - lr: 0.000006 - momentum: 0.000000
2023-10-30 16:26:09,915 epoch 2 - iter 4/6 - loss 2.02140152 - time (sec): 12.91 - samples/sec: 1207.09 - lr: 0.000007 - momentum: 0.000000
2023-10-30 16:26:12,856 epoch 2 - iter 5/6 - loss 2.00544941 - time (sec): 15.85 - samples/sec: 1242.21 - lr: 0.000008 - momentum: 0.000000
2023-10-30 16:26:14,546 epoch 2 - iter 6/6 - loss 1.97786846 - time (sec): 17.54 - samples/sec: 1276.97 - lr: 0.000008 - momentum: 0.000000
2023-10-30 16:26:14,549 ----------------------------------------------------------------------------------------------------
2023-10-30 16:26:14,552 EPOCH 2 done: 

100%|██████████| 1/1 [00:00<00:00,  1.22it/s]

...

2023-10-30 16:56:58,535 DEV : loss 1.8824336528778076 - f1-score (micro avg)  0.0218


2023-10-30 16:56:59,123 ----------------------------------------------------------------------------------------------------
2023-10-30 16:56:59,127 Testing using last state of model ...


100%|██████████| 3/3 [00:03<00:00,  1.10s/it]

2023-10-30 16:57:02,468 
Results:
- F-score (micro) 0.6274
- F-score (macro) 0.7334
- Accuracy 0.4676

By class:
              precision    recall  f1-score   support

   Component     0.6122    0.6862    0.6471       513
      Person     0.3498    0.4670    0.4000       182
        Time     0.9420    0.9701    0.9559        67
        Date     0.9216    0.9400    0.9307        50

   micro avg     0.5853    0.6761    0.6274       812
   macro avg     0.7064    0.7658    0.7334       812
weighted avg     0.5996    0.6761    0.6346       812

2023-10-30 16:57:02,469 ----------------------------------------------------------------------------------------------------


{'test_score': 0.6274285714285713}

### RE component 2

In [ ]:
relation_label_dict = corpus.make_label_dictionary(label_type="relation")
relation_dictionary = corpus.make_label_dictionary("relation")
doc_embeddings = TransformerDocumentEmbeddings(model="distilbert-base-uncased", layers="-1", fine_tune=True)
relation_model: RelationClassifier = RelationClassifier(
    embeddings =doc_embeddings,
    label_dictionary = relation_dictionary,
    label_type="relation",
    entity_label_types="ner",
    entity_pair_labels={  # Define valid entity pair combinations, used as relation candidates
        ("Person", "Component"),
        ("Person", "Time"),
        ("Person", "Date"),
        ("Person", "Person"),
        ("Component", "Person"),
        ("Component", "Date"),
        ("Component", "Time"),
        ("Component", "Component")
    },
    zero_tag_value='',
    allow_unk_tag=False,
)

2023-10-30 17:10:12,927 Computing label dictionary. Progress:


0it [00:00, ?it/s]
90it [00:00, 7395.04it/s]

2023-10-30 17:10:12,954 Dictionary created for label 'relation' with 2 values: org:happened_at (seen 355 times), org:caused_by (seen 344 times)
2023-10-30 17:10:12,956 Computing label dictionary. Progress:



0it [00:00, ?it/s]
90it [00:00, 6671.16it/s]

2023-10-30 17:10:12,983 Dictionary created for label 'relation' with 2 values: org:happened_at (seen 355 times), org:caused_by (seen 344 times)


In [ ]:
trainer: ModelTrainer = ModelTrainer(model=relation_model, corpus=relation_model.transform_corpus(corpus))
trainer.fine_tune(
    'resources/relations/',
    max_epochs=20,
    learning_rate=4e-5,
    mini_batch_size=8,
    main_evaluation_metric=("macro avg", "f1-score"),
    shuffle=True,
    embeddings_storage_mode='gpu',
)

2023-10-30 17:13:47,091 ----------------------------------------------------------------------------------------------------
2023-10-30 17:13:47,093 Model: "RelationClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768,

100%|██████████| 234/234 [00:24<00:00,  9.52it/s]


2023-10-30 17:22:03,687 DEV : loss 0.108233243227005 - f1-score (macro avg)  0.5761
2023-10-30 17:22:03,984 ----------------------------------------------------------------------------------------------------
2023-10-30 17:22:51,703 epoch 2 - iter 366/3661 - loss 0.12598204 - time (sec): 47.71 - samples/sec: 61.37 - lr: 0.000022 - momentum: 0.000000
2023-10-30 17:23:38,815 epoch 2 - iter 732/3661 - loss 0.11764908 - time (sec): 94.82 - samples/sec: 61.76 - lr: 0.000024 - momentum: 0.000000
2023-10-30 17:24:25,878 epoch 2 - iter 1098/3661 - loss 0.11674089 - time (sec): 141.89 - samples/sec: 61.91 - lr: 0.000026 - momentum: 0.000000
2023-10-30 17:25:13,234 epoch 2 - iter 1464/3661 - loss 0.10741457 - time (sec): 189.24 - samples/sec: 61.89 - lr: 0.000028 - momentum: 0.000000
2023-10-30 17:25:59,706 epoch 2 - iter 1830/3661 - loss 0.10942523 - time (sec): 235.71 - samples/sec: 62.11 - lr: 0.000030 - momentum: 0.000000
2023-10-30 17:26:47,230 epoch 2 - iter 2196/3661 - loss 0.10896951 - t

100%|██████████| 234/234 [00:05<00:00, 42.90it/s]


2023-10-30 17:30:01,298 DEV : loss 0.10641758888959885 - f1-score (macro avg)  0.5761
2023-10-30 17:30:01,770 ----------------------------------------------------------------------------------------------------
2023-10-30 17:30:48,528 epoch 3 - iter 366/3661 - loss 0.07807636 - time (sec): 46.75 - samples/sec: 62.62 - lr: 0.000040 - momentum: 0.000000
2023-10-30 17:31:36,027 epoch 3 - iter 732/3661 - loss 0.08977313 - time (sec): 94.25 - samples/sec: 62.13 - lr: 0.000040 - momentum: 0.000000
2023-10-30 17:32:22,937 epoch 3 - iter 1098/3661 - loss 0.09140590 - time (sec): 141.16 - samples/sec: 62.23 - lr: 0.000039 - momentum: 0.000000
2023-10-30 17:33:10,179 epoch 3 - iter 1464/3661 - loss 0.09155498 - time (sec): 188.41 - samples/sec: 62.16 - lr: 0.000039 - momentum: 0.000000
2023-10-30 17:33:56,820 epoch 3 - iter 1830/3661 - loss 0.09137301 - time (sec): 235.05 - samples/sec: 62.29 - lr: 0.000039 - momentum: 0.000000
2023-10-30 17:34:43,622 epoch 3 - iter 2196/3661 - loss 0.09467251 -

...

2023-10-30 19:52:34,088 DEV : loss 0.08625127375125885 - f1-score (macro avg)  0.5761


2023-10-30 19:52:35,277 ----------------------------------------------------------------------------------------------------
2023-10-30 19:52:35,279 Testing using last state of model ...


100%|██████████| 888/888 [01:34<00:00,  9.35it/s]


2023-10-30 19:54:10,474 
Results:
- F-score (micro) 0.9892
- F-score (macro) 0.8241
- Accuracy 0.9892

By class:
                 precision    recall  f1-score   support

                    0.9920    0.9969    0.9945     13866
org:happened_at     0.9720    0.7898    0.8715       176
  org:caused_by     0.6905    0.5404    0.6063       161

       accuracy                         0.9892     14203
      macro avg     0.8848    0.7757    0.8241     14203
   weighted avg     0.9884    0.9892    0.9885     14203

2023-10-30 19:54:10,475 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8240685209633786}

### Using 2-step RE on the validation set


In [3]:
loaded_ner = SequenceTagger.load("/content/drive/MyDrive/enwiki20230820/component2/ner/final-model.pt")
loaded_re: RelationClassifier = RelationClassifier.load('/content/drive/MyDrive/enwiki20230820/component2/relations/final-model.pt')

2023-11-08 11:14:33,191 SequenceTagger predicts: Dictionary with 9 tags: O, B-Component, I-Component, B-Person, I-Person, B-Time, I-Time, B-Date, I-Date


In [ ]:
test_set = srsly.read_json('/content/drive/MyDrive/enwiki20230820/component2/test_data.json')
all_relations = {}
document_dicts = []
for document in test_set:
  sentence = Sentence(document["data"]["Abstract_new"])
  loaded_ner.predict(sentence)
  loaded_re.predict(sentence)
  sent_dict = sentence.to_dict()
  # relation["labels"][0]["confidence"]
  unique_id = " ".join([str(document["data"]["EventDt"]), document["data"]["IPFCode"], document["data"]["IPFShortDesc"], str(document["inner_id"])])
  all_relations[unique_id] = [(relation["from_text"], relation["to_text"], relation["labels"][0]["value"]) for relation in sent_dict["relations"] if len(relation["labels"][0]["value"]) > 0]
  document_dicts.append(sentence.to_dict())


srsly.write_json('comp2_relations.json', all_relations)
srsly.write_json('comp2_output.json', document_dicts)
